In [ ]:
ㅇ
import numpy as np
import evaluate
import pandas as pd

In [ ]:
final_data = pd.read_csv('https://github.com/ohgzone/file1/raw/main/aihub_coupus.csv' )
final_data

,감정,문장
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.
...,...,...
51625,분노,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.
51626,불안,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.
51627,상처,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.
51628,불안,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.


In [ ]:
set(final_data['감정'].to_list())

{'기쁨', '당황', '분노', '불안', '상처', '슬픔'}

In [ ]:
text_to_label = {'기쁨': 0, '당황': 1, '분노': 2, '불안': 3, '상처': 4, '슬픔': 5}
label_to_text = {0: '기쁨', 1: '당황', 2: '분노', 3: '불안', 4: '상처', 5: '슬픔'}

In [ ]:
for key, value in text_to_label.items():
    final_data = final_data.replace({'감정': key}, value)

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(final_data)
dataset = dataset.rename_columns({'감정': 'label', '문장': 'text'})

In [ ]:
dataset

Dataset({
    features: ['label', 'text'],
    num_rows: 51630
})

In [ ]:
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM, RobertaForSequenceClassification, AutoConfig
config = AutoConfig.from_pretrained("klue/roberta-small")
config.update({"id2label": label_to_text})

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small")
model = RobertaForSequenceClassification.from_pretrained("klue/roberta-small", config=config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW
optim = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)
accuracy_metric = evaluate.load("accuracy")

for epoch in range(3):
    print(f"current epoch: {epoch}")
    model.train()
    for step, batch in enumerate(train_loader):
        optim.zero_grad()
        inputs = tokenizer(batch['text'], return_tensors='pt', padding=True, truncation=True)
        outputs = model(**inputs, labels=batch['label'])
        loss = outputs.loss
        loss.backward()
        optim.step()

        if step % 100 == 0:
            print(f"current step: {step}, loss: {loss.item()}")
    model.eval()

    references = []
    predictions = []

    for step, batch in enumerate(test_loader):
        inputs = tokenizer(batch['text'], return_tensors='pt', padding=True, truncation=True)
        predictions += model(**inputs).logits.argmax(1).tolist()
        references += batch['label'].tolist()

    results = accuracy_metric.compute(references=references, predictions=predictions)
    print(results)